In [9]:
from numba import jit
from pygraphblas import *
        
@udt
class BF:
    members = ['double w', 'uint64_t h', 'uint64_t pi']
 
@jit 
def bf_eq(x, y):
    if x.w == y.w and x.h == y.h and x.pi == y.pi:
        return True
    return False

@jit 
def bf_assign(x, y):
    x.w = y.w
    x.h = y.h
    x.pi = y.pi

@BF.binop('min')
def bf_min(z, x, y):
    if (x.w < y.w 
        or x.w == y.w and x.h < y.h 
        or x.w == y.w and x.h == y.h and x.pi < y.pi):
        if not bf_eq(z, x):
            bf_assign(z, x)
    else:
        bf_assign(z, y)    
    
@BF.binop('plus')
def bf_plus(z, x, y):
    z.w = x.w + y.w
    z.h = x.h = y.h
    if x.pi != lib.UINT64_MAX and y.pi != 0:
        z.pi = y.pi
    else:
        z.pi = x.pi
    
    
min_op = BinaryOp('min', 'BF', bf_min, BF)
plus_op = BinaryOp('plus', 'BF', bf_plus, BF)

BF.add_op = min_op
BF.mult_op = plus_op

A = Matrix.from_type(BF, 10, 10)
B = Matrix.from_type(BF, 10, 10)

A[1,1] = (0.4, 2, 9)
B[1,1] = (0.4, 2, 2)

with min_op:
    print((A + B).to_lists())

[[1], [1], [(0.4, 2, 2)]]


In [ ]:
@udt
class BF:
    members = ['double w', 'uint64_t h', 'uint64_t p']